In [1]:
# 导入标准库和第三方库
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.cuda.amp import autocast, GradScaler
import os

# 验证 mamba_ssm 导入
import mamba_ssm
print(f"mamba_ssm version: {mamba_ssm.__version__}")

# 导入同级目录的自定义模块
from mamba1_icl import Mamba1ICL
from generate_icl_data import (generate_linear_data, generate_gaussian_kernel_data,
                              generate_nonlinear_dynamical_data)

# 设置随机种子以确保可复现性
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# 确认设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 创建输出目录用于保存中间文件
output_dir = "experiment_outputs_mamba1"
os.makedirs(output_dir, exist_ok=True)

# 验证环境
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}, CUDA: {torch.version.cuda}")

mamba_ssm version: 1.2.0.post1


ImportError: cannot import name 'Mamba1ICL' from 'mamba1_icl' (/root/code/in-context-learning/mamba_icl/mamba1_icl.py)

In [ ]:
# 数据生成参数（基于提案）
num_train_prompts = 10000
num_test_prompts = 1000
context_size = 20
d = 20

# 生成数据集
print("Generating datasets...")
train_linear = generate_linear_data(num_train_prompts, context_size, d)
test_linear = generate_linear_data(num_test_prompts, context_size, d)
train_gaussian = generate_gaussian_kernel_data(num_train_prompts, context_size, d)
test_gaussian = generate_gaussian_kernel_data(num_test_prompts, context_size, d)
train_dynamical = generate_nonlinear_dynamical_data(num_train_prompts, context_size, d)
test_dynamical = generate_nonlinear_dynamical_data(num_test_prompts, context_size, d)

# 保存数据集
print("Saving datasets...")
torch.save(train_linear, os.path.join(output_dir, "train_linear.pt"))
torch.save(test_linear, os.path.join(output_dir, "test_linear.pt"))
torch.save(train_gaussian, os.path.join(output_dir, "train_gaussian.pt"))
torch.save(test_gaussian, os.path.join(output_dir, "test_gaussian.pt"))
torch.save(train_dynamical, os.path.join(output_dir, "train_dynamical.pt"))
torch.save(test_dynamical, os.path.join(output_dir, "test_dynamical.pt"))

# 验证数据集大小
print(f"Train linear prompts: {len(train_linear)}")
print(f"Test linear prompts: {len(test_linear)}")
print(f"Train gaussian prompts: {len(train_gaussian)}")
print(f"Test gaussian prompts: {len(test_gaussian)}")
print(f"Train dynamical prompts: {len(train_dynamical)}")
print(f"Test dynamical prompts: {len(test_dynamical)}")

In [ ]:
# 初始化 Mamba1 ICL 模型
model = Mamba1ICL(d_model=20, d_state=64, d_conv=4, expand=2).to(device)

# 定义优化器和混合精度训练工具
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

# 打印模型结构
print(model)

In [ ]:
# 训练参数
num_epochs = 120
batch_size = 128
datasets = [
    ("linear", train_linear),
    ("gaussian", train_gaussian),
    ("dynamical", train_dynamical)
]

# 统计训练数据分布
print("Training data distribution:")
for name, data in datasets:
    print(f"{name}: {len(data)} prompts")

# 分别训练三个模型
models = {}
context_size = 20
d = 20

for dataset_name, train_data in datasets:
    print(f"\nTraining Mamba-{dataset_name.capitalize()}...")
    model = Mamba1ICL(d_model=(d+1), d_state=64, d_conv=4, expand=2).to(device)  # d_model 调整为 d+1
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scaler = GradScaler()
    
    # 记录损失
    loss_history = []
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        num_batches = 0
        
        for i in range(0, len(train_data), batch_size):
            batch = train_data[i:i + batch_size]
            flat_prompt_batch = torch.stack([flat_prompt for flat_prompt, _, _, _, _ in batch])  # [batch, context_size*(d+1)+d]
            y_query_batch = torch.tensor([[y_query] for _, y_query, _, _, _ in batch], dtype=torch.float32).to(device)  # [batch, 1]
            X_batch = torch.stack([X for _, _, X, _, _ in batch])  # [batch, context_size, d]
            Y_batch = torch.stack([Y for _, _, _, Y, _ in batch])  # [batch, context_size]
            
            optimizer.zero_grad()
            with autocast():
                # 调整输入形状：[batch, 2*context_size+1, d+1]
                input_seq = flat_prompt_batch.view(-1, 2*context_size+1, d+1).to(device)
                output = model(input_seq, Y_batch)  # [batch, 1]
                loss = torch.nn.functional.mse_loss(output, y_query_batch)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            epoch_loss += loss.item() * len(batch)
            num_batches += len(batch)
        
        avg_loss = epoch_loss / num_batches
        loss_history.append({"epoch": epoch + 1, "loss": avg_loss})
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.6f}")
    
    # 保存损失记录
    loss_df = pd.DataFrame(loss_history)
    loss_df.to_csv(os.path.join(output_dir, f"loss_history_{dataset_name}.csv"), index=False)
    print(f"Loss history saved to {output_dir}/loss_history_{dataset_name}.csv")
    
    # 保存模型权重
    model_path = os.path.join(output_dir, f"mamba_{dataset_name}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"Model weights saved to {model_path}")
    
    models[dataset_name] = model

In [ ]:
# 保存模型权重
model_path = os.path.join(output_dir, "mamba1_icl_model.pth")
torch.save(model.state_dict(), model_path)
print(f"Model weights saved to {model_path}")

In [ ]:
from evaluate_icl import evaluate
from evaluate_i

In [ ]:
# 评估模型
print("Evaluating models with varying context sizes...")
max_context = 40
results = {"Dataset": ["Linear", "Gaussian", "Dynamical"], "Errors": []}

for dataset_name, test_data in zip(["linear", "gaussian", "dynamical"], [test_linear, test_gaussian, test_dynamical]):
    model = models[dataset_name]
    errors = evaluate_with_varying_context(model, test_data, max_context, batch_size=32, device=device)
    results["Errors"].append(errors)

# 保存评估结果
results_df = pd.DataFrame({
    "Context_Size": list(range(1, max_context + 1)),
    "Linear": results["Errors"][0],
    "Gaussian": results["Errors"][1],
    "Dynamical": results["Errors"][2]
})
results_df.to_csv(os.path.join(output_dir, "context_varying_results.csv"), index=False)
print("Context-varying results saved to experiment_outputs_mamba1/context_varying_results.csv")
print(results_df)

In [ ]:
# 绘制损失曲线
plt.figure(figsize=(10, 6))
plt.plot(loss_df["epoch"], loss_df["loss"], marker='o', label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Average MSE Loss")
plt.title("Training Loss over Epochs (Mamba1)")
plt.legend()
plt.grid(True)

# 保存图像
plot_path = os.path.join(output_dir, "loss_plot.png")
plt.savefig(plot_path)
plt.show()
print(f"Loss plot saved to {plot_path}")

In [ ]:
# 绘制误差随上下文数量变化的折线图
plt.figure(figsize=(10, 6))
plt.plot(results_df["Context_Size"], results_df["Linear"], label="Linear", marker='o')
plt.plot(results_df["Context_Size"], results_df["Gaussian"], label="Gaussian", marker='o')
plt.plot(results_df["Context_Size"], results_df["Dynamical"], label="Dynamical", marker='o')
plt.xlabel("In-Context Examples")
plt.ylabel("Squared Error")
plt.title("Squared Error vs. In-Context Examples (Mamba1)")
plt.legend()
plt.grid(True)

# 保存图像
plot_path = os.path.join(output_dir, "context_error_plot.png")
plt.savefig(plot_path)
plt.show()
print(f"Context error plot saved to {plot_path}")